# importing liberies

In [1]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Data Processing

In [2]:
df = pd.read_csv("/mnt/hdd/website_data/IMDB_dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
doc = df.review
labels = []
for i in df.sentiment:
    if i == "positive":
        labels.append(1)
    else:
        labels.append(0)

In [4]:
train_label = torch.tensor(labels, dtype = torch.float32)

In [5]:
train_label[:5]

tensor([1., 1., 1., 0., 1.])

In [6]:
tokenizer = Tokenizer(oov_token="<oov>")
tokenizer.fit_on_texts(doc)
sequences = tokenizer.texts_to_sequences(doc)
x_train = pad_sequences(sequences,padding="post")

In [7]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_train

tensor([[2.8000e+01, 5.0000e+00, 2.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [4.0000e+00, 3.9400e+02, 1.2100e+02,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.1000e+01, 1.9100e+02, 1.2000e+01,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [1.1000e+01, 2.3600e+02, 4.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.4600e+02, 1.6700e+02, 6.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [5.5000e+01, 2.8000e+01, 5.8930e+03,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [8]:
x_train.shape, train_label.shape

(torch.Size([50000, 2493]), torch.Size([50000]))

In [9]:
train_label = train_label.reshape(50000,1)
train_label.dim()

2

# Building models

In [54]:
class LLM(nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = nn.LSTM(2493,1000)
        self.lstm2 = nn.LSTM(1000, 500)

        self.dense1 = nn.Linear(500, 250)
        self.dense2 = nn.Linear(250, 125)
        self.dense3 = nn.Linear(125, 50)
        self.dense4 = nn.Linear(50,1)

    def forward(self,x):

        out,_ = self.lstm1(x)
        out,_ = self.lstm2(out)

        x = self.dense1(out)
        x = F.relu(x)
        x = self.dense2(x)
        x = F.relu(x)
        x = self.dense3(x)
        x = F.relu(x)
        x = self.dense4(x)

        return F.relu(x)

In [55]:
llm = LLM()
llm

LLM(
  (lstm1): LSTM(2493, 1000)
  (lstm2): LSTM(1000, 500)
  (dense1): Linear(in_features=500, out_features=250, bias=True)
  (dense2): Linear(in_features=250, out_features=125, bias=True)
  (dense3): Linear(in_features=125, out_features=50, bias=True)
  (dense4): Linear(in_features=50, out_features=1, bias=True)
)

In [56]:
model = torch.compile(llm)

In [57]:
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [58]:
model

OptimizedModule(
  (_orig_mod): LLM(
    (lstm1): LSTM(2493, 1000)
    (lstm2): LSTM(1000, 500)
    (dense1): Linear(in_features=500, out_features=250, bias=True)
    (dense2): Linear(in_features=250, out_features=125, bias=True)
    (dense3): Linear(in_features=125, out_features=50, bias=True)
    (dense4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [59]:
epochs = 5
for i in range(epochs):
    y_pred = model(x_train)
    loss = loss_func(y_pred,train_label)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(1.2758, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(1.0503, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.8027, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7611, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.7031, grad_fn=<BinaryCrossEntropyBackward0>)


In [36]:
model(x_train)

tensor([[0.5194, 0.4956],
        [0.5187, 0.4942],
        [0.5185, 0.4935],
        ...,
        [0.5191, 0.4954],
        [0.5188, 0.4957],
        [0.5192, 0.4952]], grad_fn=<SigmoidBackward0>)

In [60]:
model.eval()
with torch.no_grad():
    pred = model(x_train[:10])

In [62]:
pred, train_label[:10]

(tensor([[0.2573],
         [0.3213],
         [0.3811],
         [0.4197],
         [0.4345],
         [0.4459],
         [0.4420],
         [0.4476],
         [0.4340],
         [0.4154]]),
 tensor([[1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [1.]]))

# Research LSTM layer output

In [27]:
lstm_layer = nn.LSTM(2493,100)
lstm_layer2 = nn.LSTM(100, 50)
out, _= lstm_layer(x_train)
out, _= lstm_layer2(out)

In [28]:
out.shape

torch.Size([50000, 50])

In [29]:
fc = nn.Linear(50,2)

output = fc(out)

In [30]:
output

tensor([[ 0.0126, -0.0697],
        [ 0.0180, -0.1441],
        [-0.0959, -0.0897],
        ...,
        [ 0.0894, -0.1300],
        [ 0.0697, -0.1123],
        [-0.0038, -0.1127]], grad_fn=<AddmmBackward0>)